In [1]:
# %pip install -r requirements.txt
from IPython.display import clear_output
#clear_output()
from enum import Enum, auto
from dataclasses import dataclass, fields
import typing
from typing import Any, Tuple
import random
from pprint import pprint
import asyncio
import sys
import threading
import time

import ipywidgets as widgets

%load_ext mypy_ipython

In [2]:
def weighted_if(weight, out1, out2):
    return out1 if random.random() < weight else out2

In [3]:
@dataclass(frozen=True, eq=True)
class Allele:
    value: Any
    dominant: bool
    
    def mutate(self, other):
        mut = mutations.get((self, other))
        if mut is not None and random.random() < mut[1]: # mutation possible and with probability [1]
            return mut[0] # gene from mutation
        return weighted_if(0.5, self, other) # random gene if not mutated

In [4]:
class BeeGender(Enum):
    DRONE = auto()
    PRINCESS = auto()
    QUEEN = auto()

class BeeSpecies(Enum):
    FOREST      = auto()
    MEADOWS     = auto()
    COMMON      = auto()
    CULTIVATED  = auto()
    MAJESTIC    = auto()
    NOBLE       = auto()
    IMPERIAL    = auto()
    DILIGENT    = auto()
    UNWEARY     = auto()
    INDUSTRIOUS = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    # COMMON    = auto()
    # COMMON    = auto()

bs = BeeSpecies
assert len(BeeSpecies) == 10, 'Dont forget to update basic species'
basic_species = [
    bs.FOREST,
    bs.MEADOWS
]
    
local = {
    BeeGender.DRONE        : 'Drone',
    BeeGender.PRINCESS     : 'Princess',
    BeeGender.QUEEN        : 'Queen',
    BeeSpecies.FOREST      : 'FOREST',
    BeeSpecies.MEADOWS     : 'MEADOWS',    
    BeeSpecies.COMMON      : 'COMMON',     
    BeeSpecies.CULTIVATED  : 'CULTIVATED', 
    BeeSpecies.MAJESTIC    : 'MAJESTIC',   
    BeeSpecies.NOBLE       : 'NOBLE',      
    BeeSpecies.IMPERIAL    : 'IMPERIAL',   
    BeeSpecies.DILIGENT    : 'DILIGENT',   
    BeeSpecies.UNWEARY     : 'UNWEARY',    
    BeeSpecies.INDUSTRIOUS : 'INDUSTRIOUS',
    2: 2
}
    

mutations = {
    (Allele(bs.FOREST,     True), Allele(bs.MEADOWS, True)): (Allele(bs.COMMON,     True), 1),
    (Allele(bs.FOREST,     True), Allele(bs.COMMON,  True)): (Allele(bs.CULTIVATED, True), 1),
    (Allele(bs.COMMON,     True), Allele(bs.MEADOWS, True)): (Allele(bs.CULTIVATED, True), 1),
    (Allele(bs.CULTIVATED, True), Allele(bs.COMMON,  True)): (Allele(bs.MAJESTIC,   True), 1),
}
mutations.update({ (key[1], key[0]) : mutations[key] for key in mutations})
del bs

In [5]:
Gene = Tuple[Allele, Allele]

@dataclass
class Genes:
    species: Gene
    fertility: Gene
    
    def crossingover(self, genes2):
        l = []
        genes1_dict = vars(self)
        genes2_dict = vars(genes2)
        for key1, key2 in zip(genes1_dict, genes2_dict):
            assert key1 == key2, 'fields should be equal...'
            g1 = genes1_dict[key1]
            g2 = genes2_dict[key2]
            
            allele11 = weighted_if(0.5, *g1)
            allele12 = weighted_if(0.5, *g2)
            allele21 = weighted_if(0.5, *g1)
            allele22 = weighted_if(0.5, *g2)
            
            allele1 = allele11.mutate(allele12)
            allele2 = allele21.mutate(allele22)
            if not allele1.dominant and allele2.dominant:
                l.append((allele2, allele1))
            else:
                l.append((allele1, allele2))
        return Genes(*l)
    
    @staticmethod
    def sample(basic=True):
        if basic:
            species = Allele(random.sample(basic_species, 1)[0], True)
            fertility = Allele(2, True)
            return Genes(
                (species, species),
                (fertility, fertility)
            )

In [6]:
class Bee:
    def __init__(self, gender: BeeGender, genes: Genes, inspected: bool = False):
        self.gender = gender
        self.genes = genes
        self.inspected = inspected
    
    def __str__(self):
        return local[self.genes.species[0].value] + ' ' + local[self.gender]
    
    def render(self):
        if not self.inspected:
            print(self)
            return
        print(local[self.gender])
        genes = vars(self.genes)
        for key in genes:
            print(key, ':', local[genes[key][0].value], 'dom:', genes[key][0].dominant, local[genes[key][1].value], 'dom:', genes[key][0].dominant)

class Queen(Bee):
    def __init__(self, g1, g2, lifespan, inspected: bool = False):
        self.g1 = g1
        self.g2 = g2
        g = self.g1.crossingover(self.g2)
        super().__init__(BeeGender.QUEEN, g, inspected)
        self.lifespan = lifespan
        self.remaining_lifespan = lifespan
    
    def die(self):
        return [Princess(self.genes)] + [Drone(self.g1.crossingover(self.g2)) for i in range(self.genes.fertility[0].value)]
    
    def render(self):
        super().render()
        print('ticks remaining:', self.remaining_lifespan)
    
class Princess(Bee):
    def __init__(self, genes, inspected: bool = False):
        super().__init__(BeeGender.PRINCESS, genes, inspected)
    
    def mate(self, other: 'Drone') -> Queen:
        if other.gender is not BeeGender.DRONE:
            raise TypeError('Princesses can only mate drones')
        return Queen(self.genes, other.genes, 3, True)

class Drone(Bee):
    def __init__(self, genes, inspected: bool = False):
        super().__init__(BeeGender.DRONE, genes, inspected)

In [7]:
pf = Princess(Genes(
        (Allele(BeeSpecies.FOREST, True), Allele(BeeSpecies.FOREST, True)),
        (Allele(2, True), Allele(2, True)),
    ))
dm = Drone(Genes(
        (Allele(BeeSpecies.MEADOWS, True), Allele(BeeSpecies.MEADOWS, True)),
        (Allele(2, True), Allele(2, True)),
    ))
qc = pf.mate(dm)
pc, dc, dc2 = qc.die()
qcu = pc.mate(dm)
pcu, dcu, dcu2 = qcu.die()
pcu.mate(dc).render()

Queen
species : MAJESTIC dom: True MAJESTIC dom: True
fertility : 2 dom: True 2 dom: True
ticks remaining: 3


In [8]:
class Inventory:
    def __init__(self, capacity=None):
        self.capacity = capacity or 100
        self.storage = [None] * self.capacity
        
    def __setitem__(self, key, value):
        if self.storage[key] is None:
            self.storage[key] = value
        else:
            raise IndexError('The slot is not empty')
    
    def __getitem__(self, key):
        return self.storage[key]
    
    def __str__(self):
        res = ['------ INV ------']
        
        empty = True
        for index, bee in enumerate(self.storage):
            if bee is not None:
                empty = False
                res.append(str(index) + ' ' + str(bee))
        if empty:
            res.append('Empty...')
        
        return '\n'.join(res)
    
    def swap(self, i1, i2):
        self.storage[i1], self.storage[i2] = self.storage[i2], self.storage[i1]
        
    def mate(self, i1, i2):
        self.storage[i1] = self.storage[i1].mate(self.storage[i2])
        self.storage[i2] = None
        
    def render(self):
        print(self)
                
    def place_bees(self, offspring, parent_index=None):
        if parent_index is not None:
            self.storage[parent_index] = None
        index = 0
        for bee in offspring:
            while index < self.capacity:
                try:
                    self.__setitem__(index, bee)
                    index += 1
                    break
                except IndexError:
                    index += 1
                    continue
            else:
                print('Beware that', bee, 'was thrown out...')
    

In [9]:
inv = Inventory(100)
inv[0] = pf
inv[1] = dm
inv.render()
inv.mate(0, 1)
inv.render()
inv.place_bees(inv[0].die(), 0)
inv.render()

------ INV ------
0 FOREST Princess
1 MEADOWS Drone
------ INV ------
0 COMMON Queen
------ INV ------
0 COMMON Princess
1 COMMON Drone
2 COMMON Drone


In [10]:
def forage():
    genes = Genes.sample()
    print(type(genes))
    return Princess(genes), Drone(genes)

async def ainput(string: str ='') -> str:
    await asyncio.get_event_loop().run_in_executor(
            None, lambda s=string: sys.stdout.write(s+' '))
    return await asyncio.get_event_loop().run_in_executor(
            None, sys.stdin.readline)

In [27]:
class CLI:
    def __init__(self):
        #command, *params = (input()).split()
        self.command_lock = threading.Lock()
        self.command_event = threading.Event()
        self.exit_event = threading.Event()
        self.command = None
        self.params = None
        
        self.out = widgets.Output()
        self.text = widgets.Text()
        self.button = widgets.Button(icon='check')
        self.button.on_click(self.register_command)
        display(widgets.VBox([self.out, self.text, self.button]))

        
        self.inner_state_thread = threading.Thread(target=self.update_state)
        self.inner_state_thread.start()
        
    def print(self, *strings, sep=' ', end='\n'):
        self.out.append_stdout(sep.join(map(str, strings)) + end)
        
    def register_command(self, b):
        self.print('much wow', self.text.value)
        if self.exit_event.is_set():
            return
        self.print('just in case', self.text.value)
        command, *params = self.text.value.split()
        with self.command_lock:
            self.command, self.params = command, params
            self.command_event.set()
            self.print('set event')
    
    def update_state(self):
        self.print('why nothing works?')
        inv = Inventory(100)
        honey = 0
        while True:
            time.sleep(2)
            self.out.outputs = tuple()
            honey += 1
            self.print(honey)
            if self.command_event.is_set():
                with self.command_lock:
                    if self.command in ['exit', 'q']:
                        self.exit_event.set()
                        break
                    elif self.command == 'inv':
                        if len(self.params) == 0:
                            self.print(inv)
                        else:
                            try:
                                inv[int(self.params[0])].render()
                            except AttributeError:
                                self.print('Slot empty')
                    elif self.command == 'mate':
                        try:
                            inv.mate(*map(int, self.params))
                        except TypeError as e:
                            print(e)
                    elif self.command == 'swap':
                        inv.swap(*map(int, self.params))
                    elif self.command == 'forage':
                        res = forage()
                        inv.place_bees(res)

                    self.command_event.clear()
                    
        self.print('Exiting...')

In [28]:
cli = CLI()

In [13]:
1/0

ZeroDivisionError: division by zero

In [ ]:
from IPython.display import display
event = threading.Event()
out = widgets.Output()
text = widgets.Text()
button = widgets.Button(icon='check')

def print_my(b):
    if text.value == 'disarm':
        other_function(out)
        event.set()
    else:
        out.append_stdout('not disarm\n')

button.on_click(print_my)
    
display(widgets.VBox([out, text, button]))

def background(out):
    while True:
        time.sleep(3)
        if event.is_set():
            break
        out.outputs = tuple()
        out.append_stdout('disarm me by typing disarm\n')


def other_function(out):
        out.append_stdout('You disarmed me! Dying now.\n')

# now threading1 runs regardless of user input
threading1 = threading.Thread(target=background, args=(out,))
threading1.start()